# <center>Тестинг функций и методов для торгового бота
Статью по написанию бота см. здесь:   
https://quantrum.me/2796-prostoj-bot-dlya-ib/   
С полной версией кода по статье можно ознакомиться здесь:   
https://gitlab.com/dmvlch/smaint/blob/master/SMAint.ipynb   

В настоящем ноутбуке тестировались функции по мере прочтения этой статьи

In [15]:
import requests
import pandas as pd
import time
import talib
import pytz
import datetime
import os
from ib_insync import *

In [2]:
ticker = str(input('Enter ticker: ')).upper()
count = int(input('Amount: '))

Enter ticker:  aapl
Amount:  30


In [3]:
ticker

'AAPL'

In [4]:
contract = Stock(ticker)
contract

Stock(symbol='AAPL')

## Получение минутных данных
Загрузка данных осуществляется с помощью сервиса [Alpha Vantage](https://www.alphavantage.co/). Здесь, с определенными ограничениями, можно бесплатно взять биржевые данные американских компаний.   

### Функция загрузки данных

In [5]:
def load_data(ticker, path='data', interval='1min', apikey='AO58MG05QIL3QRN2'):
    filename = f'{ticker}.csv'
    path_to_file = os.path.join(path, filename)
    url = f'https://www.alphavantage.co/query?'\
          f'function=TIME_SERIES_INTRADAY&symbol={ticker}&'\
          f'interval={interval}&apikey={apikey}&datatype=csv'
    r = requests.get(url)
    content = r.content.decode('UTF-8')
    with open(path_to_file, "w") as file:
        file.write(content)
    print(f'\tUpdate {ticker} price history:', path_to_file)
    return path_to_file

In [6]:
path_to_file = load_data(ticker)

	Update AAPL price history: data\AAPL.csv


In [7]:
data = pd.read_csv(path_to_file)
data.head()

,timestamp,open,high,low,close,volume
0,2020-09-11 20:00:00,111.96,112.00,111.96,111.99,9351
1,2020-09-11 19:59:00,111.96,111.99,111.95,111.99,7252
2,2020-09-11 19:58:00,111.93,111.96,111.93,111.95,5995
3,2020-09-11 19:57:00,111.95,111.97,111.94,111.96,6783
4,2020-09-11 19:56:00,111.95,111.97,111.94,111.97,2830


## Вспомогательные методы
### Функция получения часового пояса

In [11]:
def get_tz(ticker, interval = '1min', apikey = 'AO58MG05QIL3QRN2'):
    filename = f'{ticker}.json'
    url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={ticker}&interval={interval}&apikey={apikey}'
    r = requests.get(url)
    content = r.content.decode('UTF-8')
    with open(filename, "w") as file:
        file.write(content)
    data = pd.read_json(f'{ticker}.json')
    tz = data['Meta Data']['6. Time Zone']
    os.remove(f'{ticker}.json')
 
    return tz

In [9]:
tz = get_tz(ticker)
tz

'US/Eastern'

### функции time_check, которая:
- определит торговые дни и часы;
- разделит торговые часы на состояния рынка (премаркет, регулярная сессия, постмаркет);
- определит местное время (для этого прежде получали часовой пояс);
- сопоставит местное время с состоянием рынка;
- вернет текущее состояние рынка.

In [5]:
def time_check(ticker):
    contract = Stock(ticker)
    cds = ib.reqContractDetails(contract)
    hours = cds[0].tradingHours
    hourslist1 = hours.split(';')
    hourslist2 = hourslist1[0].split('-')
    hourslistopening = hourslist2[0].split(':')
    tz = get_tz(ticker)
    today = datetime.datetime.now(tz=pytz.UTC).astimezone(pytz.timezone(tz))
    date = today.strftime("%Y%m%d")
    time = today.strftime("%H%M")
    print('DateTime: ', today.strftime("%d-%m-%Y at %I:%M%p"), tz)
    if hourslistopening[1] == 'CLOSED':
        return 'close'
    else:
        hourslistclosing = hourslist2[1].split(':')
        openingsdict = dict(zip(hourslistopening[::2], hourslistopening[1::2]))
        closingdict = dict(zip(hourslistclosing[::2], hourslistclosing[1::2]))
        hoursregular = cds[0].liquidHours
        hourslist1regular = hoursregular.split(';')
        hourslist2regular = hourslist1regular[0].split('-')
        hourslistopeningregular = hourslist2regular[0].split(':')
        hourslistclosingregular = hourslist2regular[1].split(':')
        openingsdictregular = dict(zip(hourslistopeningregular[::2], hourslistopeningregular[1::2]))
        closingdictregular = dict(zip(hourslistclosingregular[::2], hourslistclosingregular[1::2]))
        rangelist = []
        for key, value in openingsdict.items():
            rangelist.append(value)
        for key, value in closingdict.items():
            rangelist.append(value)
        for key, value in openingsdictregular.items():
            rangelist.append(value)
        for key, value in closingdictregular.items():
            rangelist.append(value)
        sortrangelist = sorted(rangelist)
        if sortrangelist[0] <= time < sortrangelist[1]:
            return 'premarket'
        elif sortrangelist[1] <= time < sortrangelist[2]:
            return 'regular'
        elif sortrangelist[2] <= time < sortrangelist[3]:
            return 'postmarket'
        else:
            return 'close'

### Функции коннекта и дисконнекта с биржей через IB

In [6]:
def connect():
    if not ib.isConnected():  # connect only without active connection    
        try:
            print('Connecting...')
            ib.connect('127.0.0.1', 7497, clientId=1)  # connect
        except Exception as ex:
            print('Error:', ex)  # catch at exception

    if ib.isConnected():
        print("Successfully connected")

In [7]:
def disconnect():
    if ib.isConnected():  # check for connection
        ib.disconnect()  # disconnect

        while ib.isConnected():  # wait while disconnecting
            time.sleep(1)  # sleep 1 sec on waiting

    print("Successful disconnected with TWS")

In [16]:
util.startLoop()
ib = IB()

In [18]:
connect()

Connecting...
Successfully connected


In [19]:
time_check(ticker)

DateTime:  14-09-2020 at 04:18AM US/Eastern


'premarket'

In [20]:
# disconnect()

### Проверка баланса и позиций

In [21]:
def balance():
    balances = {av.tag: float(av.value) for av in ib.accountSummary() if av.tag in ['AvailableFunds', 'BuyingPower', 'TotalCashValue', 'NetLiquidation']}
    balance = balances.get('AvailableFunds', 0)
 
    return balance

In [22]:
balance()

1219992.2

### Проверка покупательной способности с помощью функции **check_balance**

In [53]:
def read_data(ticker, path='data'):
    filename = f'{ticker}.csv'
    path_to_file = os.path.join(path, filename)
    data = pd.read_csv(path_to_file)
    os.remove(path_to_file)
    
    return data

In [23]:
def check_balance(ticker, count):
    price = read_data(ticker)['close'].iloc[-1]
    amount = price * count
    if balance() > amount:
        return True
    else:
        return False

In [30]:
check_balance(ticker, count)

True

### список открытых позиций, получаемый с помощью метода **positions** из IB.API

In [31]:
def list_positions():
    positions = ib.positions()
    positions = "\n".join([f"{p.contract.localSymbol} {p.position}x{p.avgCost}" for p in positions])
 
    return positions

Данный код возвращает наименование тикера, количество и стоимость каждой позиции

In [33]:
print(list_positions())

ESGV 1142.0x52.7765556
ESGD 954.0x64.1715282
DVY 292.0x101.06176745
VO 50.0x170.48881
VDE 99.0x81.962902
TFI 1109.0x50.50644005
IBKR 2800.0x39.4058383
TIP 207.0x115.00465265
BWX 712.0x28.57725195
LQD 99.0x124.4389121
ESML 309.0x27.38696345
VNQ 137.0x88.7322321
ESGE 1221.0x33.994677
EAGG 1280.0x53.3022901


### функция **list_orders**, показывающая список **активных ордеров** использующая метод **openTrades**

In [34]:
def list_orders():
    trades = ib.openTrades()
    orders = "\n".join([f"{t.order.action} {t.contract.secType} {t.contract.symbol} {t.contract.localSymbol}" f" {t.order.totalQuantity}x{t.order.lmtPrice}" for t in trades])
 
    return orders

In [35]:
list_orders()

''

## Описание логики пересечений
Расчет скользящих средних, реализацию логики пересечений и подачу сигнала на покупку/продажу осуществляет функция **algorithm**

Получим цены закрытия

In [36]:
try:
    close = read_data(ticker)['close'].fillna(method='ffill')
except KeyError:
    print('No data, the limit of requests may be exceeded')
    disconnect()

In [37]:
close

0     111.99
1     111.99
2     111.95
3     111.96
4     111.97
       ...  
95    112.00
96    112.01
97    112.04
98    112.01
99    112.02
Name: close, Length: 100, dtype: float64

In [38]:
ma_long = talib.SMA(close, timeperiod=26)
ma_short = talib.SMA(close, timeperiod=9)

In [40]:
print(ma_long, ma_short)

0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
95    111.962308
96    111.963462
97    111.966923
98    111.970385
99    111.973077
Length: 100, dtype: float64 0            NaN
1            NaN
2            NaN
3            NaN
4            NaN
         ...    
95    111.990000
96    111.991111
97    111.995556
98    111.997778
99    112.003333
Length: 100, dtype: float64


если хотим скачать более 100 последних значений

In [43]:
interval='1min'; apikey='AO58MG05QIL3QRN2'
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&'\
      f'symbol={ticker}&interval={interval}&apikey={apikey}&datatype=csv&outputsize=full'
url

'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol=AAPL&interval=1min&apikey=AO58MG05QIL3QRN2&datatype=csv&outputsize=full'

для приказа на сделку 9покупку или продажу) учитываем противоположность сигналов за последнюю и предыдущую минуту

In [44]:
if ma_long.iloc[-2] > ma_short.iloc[-2] and ma_long.iloc[-1] < ma_short.iloc[-1]:
    contract = contract(ticker)
    buy(ticker, contract)
elif ma_long.iloc[-2] < ma_short.iloc[-2] and ma_long.iloc[-1] > ma_short.iloc[-1]:
    contract = contract(ticker)
    sell(ticker, contract)

In [45]:
def buy(ticker, contract):
    if f'{ticker}' not in list_positions():
        if f'{ticker}' not in list_orders():
            order = MarketOrder('BUY', f'{count}')
            ib.placeOrder(contract, order)

In [46]:
def sell(ticker, contract):
    if f'{ticker}' in list_positions():
        if f'{ticker}' not in list_orders(): 
            order = MarketOrder('SELL', f'{count}')
            ib.placeOrder(contract, order)

#### Код функции **algorithm** полностью:

In [47]:
def algorithm(ticker):
    try:
        close = read_data(ticker)['close'].fillna(method='ffill')
    except KeyError:
        print('No data, the limit of requests may be exceeded')
        disconnect()
    else:
        ma_long = talib.SMA(close, timeperiod=26)
        ma_short = talib.SMA(close, timeperiod=9)
        last_price = close.iloc[-1]
        print(f'\tCheck SMA for {ticker} (last={last_price:.2f}): Short={ma_short.iloc[-1]:.4f} Long={ma_long.iloc[-1]:.4f}')
        print(f'\tCheck SMA for {ticker} (last={last_price:.2f}): Short={ma_short.iloc[-2]:.4f} Long={ma_long.iloc[-2]:.4f} (previous)')
        if ma_long.iloc[-2] > ma_short.iloc[-2] and ma_long.iloc[-1] < ma_short.iloc[-1]:
            contract = trade(ticker)
            buy(ticker, contract)
        elif ma_long.iloc[-2] < ma_short.iloc[-2] and ma_long.iloc[-1] > ma_short.iloc[-1]:
            contract = trade(ticker)
            sell(ticker, contract)

## Основной цикл работы

In [48]:
# while True:
#     load_data(ticker)
#     algorithm(ticker)
#     time.sleep(60)

In [49]:
# time_check()
# check_balance()

В случае прохождения всех проверок работу начинает основной цикл. Все это осуществляет функция **run_algorithm**

In [54]:
util.startLoop()
ib = IB()

In [55]:
connect()

Connecting...
Successfully connected


In [50]:
def run_algorithm():
    messages = {
      'premarket': 'Premarket',
      'postmarket': 'Postmarket',
      'close': 'Closed',
      'regular': 'Regular session', }
    message = messages.get(time_check(ticker), 'Uncertain')
    print(message)
    if message == 'Regular session':
        load_data(ticker)
        if check_balance(ticker, count):
            print('== Start working ==')
            while True:
                load_data(ticker)
                algorithm(ticker)
                time.sleep(60)
        else:
            print('Insufficient funds')
    else:
        print('Wait for the regular session to open')

In [56]:
run_algorithm()

DateTime:  14-09-2020 at 06:06AM US/Eastern
Premarket
Wait for the regular session to open


In [57]:
disconnect()

Successful disconnected with TWS
